In [5]:
suppressMessages(library(tidyverse))
suppressMessages(library(data.table))

In [45]:
suppressMessages(library(DESeq2))

In [12]:
getwd()

[1] "/project2/yangili1/cdai/SpliFi/analysis"

In [13]:
setwd('../code/test-ge')

In [14]:
dir()

[1] "Kidney-Cortex_gene_reads.tsv.gz" "Liver_gene_reads.tsv.gz"

In [190]:
'abc/def/Kidney-Cortex_gene_reads.tsv.gz' %>% 
str_split("/") %>% unlist %>% tail(1) %>% 
str_remove("_gene_reads.tsv.gz")

[1] "Kidney-Cortex"

### Preparing input data

In [16]:
kidney <- fread('Kidney-Cortex_gene_reads.tsv.gz')
liver <- fread('Liver_gene_reads.tsv.gz')

In [17]:
dim(kidney)
dim(liver)

[1] 56200    87

[1] 56200   228

In [18]:
# use the first 5 samples of each as test data

fwrite(kidney[, 1:12], 'Kidney-Cortex_gene_reads.test.tsv.gz')
fwrite(liver[, 1:12], 'Liver_gene_reads.test.tsv.gz')

### DeSeq2 requirements:

count matrix:
1. needs to have row names. row names are genes (must be unique, best use gene_id)
2. columns are counts, column names are sample names, must be unique

column data: 
1. needs to have row names.row names are column names in the exact order as in count matrix
2. columns are conditions, and other groups or classification of samples. Used for contrasts

In [191]:
ncol(kidney)

[1] 87

In [19]:
kidney[1:5, 1:5]
liver[1:5, 1:5]

Name,Description,GTEX-11GS4-2326-SM-5A5KS,GTEX-11OF3-1326-SM-5N9FJ,GTEX-11TTK-1926-SM-5PNW8
<chr>,<chr>,<int>,<int>,<int>
ENSG00000223972.5,DDX11L1,0,3,0
ENSG00000227232.5,WASH7P,183,149,182
ENSG00000278267.1,MIR6859-1,0,0,0
ENSG00000243485.5,MIR1302-2HG,1,2,0
ENSG00000237613.2,FAM138A,2,2,0


Name,Description,GTEX-11DXY-0526-SM-5EGGQ,GTEX-11DXZ-0126-SM-5EGGY,GTEX-11EQ9-0526-SM-5A5JZ
<chr>,<chr>,<int>,<int>,<int>
ENSG00000223972.5,DDX11L1,1,0,1
ENSG00000227232.5,WASH7P,81,69,39
ENSG00000278267.1,MIR6859-1,0,0,0
ENSG00000243485.5,MIR1302-2HG,1,1,1
ENSG00000237613.2,FAM138A,0,0,0


In [38]:
counts <- cbind(
    # take 5 samples from kidney, and gene id column as first column
    kidney[, c(1, 3:7)],
    # take 5 samples from liver, sample columns only
    liver[, c(3:7)]
)

In [39]:
# convert gene_id column into row ids
counts <- column_to_rownames(counts, "Name")

In [40]:
counts[1:5,]
dim(counts)

,GTEX-11GS4-2326-SM-5A5KS,GTEX-11OF3-1326-SM-5N9FJ,GTEX-11TTK-1926-SM-5PNW8,GTEX-12696-0926-SM-5FQTV,GTEX-12WSG-0826-SM-5EQ5A,GTEX-11DXY-0526-SM-5EGGQ,GTEX-11DXZ-0126-SM-5EGGY,GTEX-11EQ9-0526-SM-5A5JZ,GTEX-11GSP-0626-SM-5986T,GTEX-11NUK-1226-SM-5P9GM
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
ENSG00000223972.5,0,3,0,0,2,1,0,1,3,0
ENSG00000227232.5,183,149,182,104,31,81,69,39,56,61
ENSG00000278267.1,0,0,0,0,0,0,0,0,0,0
ENSG00000243485.5,1,2,0,1,0,1,1,1,0,1
ENSG00000237613.2,2,2,0,0,0,0,0,0,0,0


[1] 56200    10

In [41]:
coldata <- data.frame(sample_name = colnames(counts), tissue = c(rep('Kidney', 5), rep('Liver', 5)))

In [42]:
coldata

sample_name,tissue
<chr>,<chr>
GTEX-11GS4-2326-SM-5A5KS,Kidney
GTEX-11OF3-1326-SM-5N9FJ,Kidney
GTEX-11TTK-1926-SM-5PNW8,Kidney
GTEX-12696-0926-SM-5FQTV,Kidney
GTEX-12WSG-0826-SM-5EQ5A,Kidney
GTEX-11DXY-0526-SM-5EGGQ,Liver
GTEX-11DXZ-0126-SM-5EGGY,Liver
GTEX-11EQ9-0526-SM-5A5JZ,Liver
GTEX-11GSP-0626-SM-5986T,Liver


In [145]:
# now construct a deseq dataset using counts and coldata
dds <- DESeqDataSetFromMatrix(
    countData = counts,
    colData = coldata,
    design = ~tissue
)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


In [146]:
dds$tissue <- factor(dds$tissue, levels = c('Kidney', 'Liver'))

In [147]:
# add gene_name as feature
mcols(dds) <- DataFrame(mcols(dds), data.frame(gene_name = kidney$Description))

In [149]:
# filter: minimum 3 groups with at least 10 reads each
min_group_size <- 3
min_read <- 10
keep_genes <- rowSums(counts(dds) >= min_read) >= min_group_size

In [150]:
str(keep_genes)
sum(keep_genes)

 Named logi [1:56200] FALSE TRUE FALSE FALSE FALSE FALSE ...
 - attr(*, "names")= chr [1:56200] "ENSG00000223972.5" "ENSG00000227232.5" "ENSG00000278267.1" "ENSG00000243485.5" ...


[1] 22525

In [151]:
# remaining counts
dds <- dds[keep_genes,]

In [152]:
dds$tissue

[1] Kidney Kidney Kidney Kidney Kidney Liver  Liver  Liver  Liver  Liver 
Levels: Kidney Liver

In [153]:
# build model
dds <- DESeq(dds)
res <- results(dds)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [154]:
resultsNames(dds)

[1] "Intercept"              "tissue_Liver_vs_Kidney"

In [155]:
results(dds)

log2 fold change (MLE): tissue Liver vs Kidney 
Wald test p-value: tissue Liver vs Kidney 
DataFrame with 22525 rows and 6 columns
                     baseMean log2FoldChange     lfcSE       stat     pvalue
                    <numeric>      <numeric> <numeric>  <numeric>  <numeric>
ENSG00000227232.5     88.4724     -0.0966055  0.455321 -0.2121699 0.83197449
ENSG00000268903.1     60.1416     -0.0200710  0.631293 -0.0317935 0.97463676
ENSG00000269981.1     35.0510     -0.0047140  0.654322 -0.0072044 0.99425177
ENSG00000241860.6      9.3820     -1.9187331  0.731552 -2.6228265 0.00872037
ENSG00000279457.4    217.5392     -0.1603465  0.476790 -0.3363043 0.73664138
...                       ...            ...       ...        ...        ...
ENSG00000210184.1      11.477      -3.065321  1.171764  -2.615989 0.00889694
ENSG00000198786.2  504948.887       1.063829  0.617588   1.722555 0.08496900
ENSG00000198695.2  149161.431       1.181671  0.682817   1.730582 0.08352633
ENSG00000210194.1     

In [133]:
results(dds, contrast = c('tissue',  'Kidney', 'Liver'))

log2 fold change (MLE): tissue Kidney vs Liver 
Wald test p-value: tissue Kidney vs Liver 
DataFrame with 22525 rows and 6 columns
                     baseMean log2FoldChange     lfcSE      stat     pvalue
                    <numeric>      <numeric> <numeric> <numeric>  <numeric>
ENSG00000227232.5     88.4724      0.0966055  0.455321 0.2121699 0.83197449
ENSG00000268903.1     60.1416      0.0200710  0.631293 0.0317935 0.97463676
ENSG00000269981.1     35.0510      0.0047140  0.654322 0.0072044 0.99425177
ENSG00000241860.6      9.3820      1.9187331  0.731552 2.6228265 0.00872037
ENSG00000279457.4    217.5392      0.1603465  0.476790 0.3363043 0.73664138
...                       ...            ...       ...       ...        ...
ENSG00000210184.1      11.477       3.065321  1.171764  2.615989 0.00889694
ENSG00000198786.2  504948.887      -1.063829  0.617588 -1.722555 0.08496900
ENSG00000198695.2  149161.431      -1.181671  0.682817 -1.730582 0.08352633
ENSG00000210194.1     172.203    

In [68]:
res

log2 fold change (MLE): tissue Liver vs Kidney 
Wald test p-value: tissue Liver vs Kidney 
DataFrame with 22525 rows and 6 columns
                     baseMean log2FoldChange     lfcSE       stat     pvalue
                    <numeric>      <numeric> <numeric>  <numeric>  <numeric>
ENSG00000227232.5     88.4724     -0.0966055  0.455321 -0.2121699 0.83197449
ENSG00000268903.1     60.1416     -0.0200710  0.631293 -0.0317935 0.97463676
ENSG00000269981.1     35.0510     -0.0047140  0.654322 -0.0072044 0.99425177
ENSG00000241860.6      9.3820     -1.9187331  0.731552 -2.6228265 0.00872037
ENSG00000279457.4    217.5392     -0.1603465  0.476790 -0.3363043 0.73664138
...                       ...            ...       ...        ...        ...
ENSG00000210184.1      11.477      -3.065321  1.171764  -2.615989 0.00889694
ENSG00000198786.2  504948.887       1.063829  0.617588   1.722555 0.08496900
ENSG00000198695.2  149161.431       1.181671  0.682817   1.730582 0.08352633
ENSG00000210194.1     

In [72]:
resOrdered <- res[order(res$pvalue),]

In [73]:
resOrdered

log2 fold change (MLE): tissue Liver vs Kidney 
Wald test p-value: tissue Liver vs Kidney 
DataFrame with 22525 rows and 6 columns
                    baseMean log2FoldChange     lfcSE      stat       pvalue
                   <numeric>      <numeric> <numeric> <numeric>    <numeric>
ENSG00000175003.12   26377.9        10.2582  0.290184   35.3506 9.80489e-274
ENSG00000138115.13   45803.6        10.6983  0.308490   34.6795 1.60370e-263
ENSG00000166589.12   19925.5       -11.5025  0.376032  -30.5891 1.71007e-205
ENSG00000158874.11  105569.7        16.0543  0.532977   30.1220 2.49720e-199
ENSG00000141505.11   21567.0        10.3431  0.346302   29.8673 5.23102e-196
...                      ...            ...       ...       ...          ...
ENSG00000272733.1    85.6033      -2.051008  0.826876  -2.48043           NA
ENSG00000100170.9   257.9066       3.368497  0.577058   5.83736           NA
ENSG00000188064.9   131.5914      -3.840662  1.138880  -3.37232           NA
ENSG00000008056.13   7

In [86]:
res[1:5,]

log2 fold change (MLE): tissue Liver vs Kidney 
Wald test p-value: tissue Liver vs Kidney 
DataFrame with 5 rows and 6 columns
                   baseMean log2FoldChange     lfcSE       stat     pvalue
                  <numeric>      <numeric> <numeric>  <numeric>  <numeric>
ENSG00000227232.5   88.4724     -0.0966055  0.455321 -0.2121699 0.83197449
ENSG00000268903.1   60.1416     -0.0200710  0.631293 -0.0317935 0.97463676
ENSG00000269981.1   35.0510     -0.0047140  0.654322 -0.0072044 0.99425177
ENSG00000241860.6    9.3820     -1.9187331  0.731552 -2.6228265 0.00872037
ENSG00000279457.4  217.5392     -0.1603465  0.476790 -0.3363043 0.73664138
                       padj
                  <numeric>
ENSG00000227232.5 0.8873442
ENSG00000268903.1 0.9847829
ENSG00000269981.1 0.9962817
ENSG00000241860.6 0.0224058
ENSG00000279457.4 0.8173485

In [159]:
(res$padj < .1 & abs(res$log2FoldChange) > 1) %>% str

 logi [1:22525] FALSE FALSE FALSE TRUE FALSE FALSE ...


In [109]:
sig.rows <- which(res$padj < .1 & (abs(res$log2FoldChange) > 1))

In [110]:
summary(sig.rows)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
      4    5504   11052   11108   16673   22521 

In [111]:
res[sig.rows,]

log2 fold change (MLE): tissue Liver vs Kidney 
Wald test p-value: tissue Liver vs Kidney 
DataFrame with 8810 rows and 6 columns
                     baseMean log2FoldChange     lfcSE      stat      pvalue
                    <numeric>      <numeric> <numeric> <numeric>   <numeric>
ENSG00000241860.6      9.3820       -1.91873  0.731552  -2.62283 0.008720367
ENSG00000229376.3     16.5887        1.86375  0.642316   2.90161 0.003712534
ENSG00000228794.8   1334.6976        1.27892  0.365885   3.49543 0.000473302
ENSG00000272438.1     11.5670       -2.11203  0.831783  -2.53916 0.011111984
ENSG00000230699.2     35.7945       -1.36888  0.522714  -2.61880 0.008824030
...                       ...            ...       ...       ...         ...
ENSG00000210077.1 6.16407e+00       -2.29060  0.957939  -2.39118  0.01679438
ENSG00000209082.1 6.35123e+01       -2.25402  0.731392  -3.08183  0.00205735
ENSG00000210144.1 1.04810e+01        2.15472  0.847218   2.54328  0.01098158
ENSG00000198804.2 2.499

In [114]:
res.sig <- res[sig.rows,]

In [116]:
res.sig <- res.sig[order(-abs(res.sig$log2FoldChange)),]

In [117]:
res.sig

log2 fold change (MLE): tissue Liver vs Kidney 
Wald test p-value: tissue Liver vs Kidney 
DataFrame with 8810 rows and 6 columns
                     baseMean log2FoldChange     lfcSE      stat       pvalue
                    <numeric>      <numeric> <numeric> <numeric>    <numeric>
ENSG00000080910.11    7417.58        16.7420  0.973718   17.1939  2.94869e-66
ENSG00000021852.12   14169.08        16.5237  1.014426   16.2887  1.18666e-59
ENSG00000165471.6     6345.60        16.5169  1.068013   15.4650  5.97322e-54
ENSG00000158874.11  105569.72        16.0543  0.532977   30.1220 2.49720e-199
ENSG00000101981.10   15902.07        15.7815  0.819170   19.2653  1.05119e-82
...                       ...            ...       ...       ...          ...
ENSG00000087460.24 27228.2080       -1.00074  0.241870  -4.13753  3.51068e-05
ENSG00000272599.2     43.4623        1.00042  0.438798   2.27992  2.26126e-02
ENSG00000184619.3    102.1086       -1.00011  0.410550  -2.43602  1.48497e-02
ENSG00000223

In [118]:
summary(res.sig)


out of 8810 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)       : 3984, 45%
LFC < 0 (down)     : 4826, 55%
outliers [1]       : 0, 0%
low counts [2]     : 0, 0%
(mean count < 3)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results



In [119]:
getwd()

[1] "/project2/yangili1/cdai/SpliFi/code/test-ge"

In [120]:
write.csv(res.sig, "results.csv")

In [121]:
str(res.sig)

Formal class 'DESeqResults' [package "DESeq2"] with 7 slots
  ..@ priorInfo      : list()
  ..@ rownames       : chr [1:8810] "ENSG00000080910.11" "ENSG00000021852.12" "ENSG00000165471.6" "ENSG00000158874.11" ...
  ..@ nrows          : int 8810
  ..@ elementType    : chr "ANY"
  ..@ elementMetadata:Formal class 'DFrame' [package "S4Vectors"] with 6 slots
  .. .. ..@ rownames       : NULL
  .. .. ..@ nrows          : int 6
  .. .. ..@ elementType    : chr "ANY"
  .. .. ..@ elementMetadata: NULL
  .. .. ..@ metadata       : list()
  .. .. ..@ listData       :List of 2
  .. .. .. ..$ type       : chr [1:6] "intermediate" "results" "results" "results" ...
  .. .. .. ..$ description: chr [1:6] "mean of normalized counts for all samples" "log2 fold change (MLE): tissue Liver vs Kidney" "standard error: tissue Liver vs Kidney" "Wald statistic: tissue Liver vs Kidney" ...
  ..@ metadata       :List of 6
  .. ..$ filterThreshold: Named num 2.64
  .. .. ..- attr(*, "names")= chr "0%"
  .. ..$ fi

In [163]:
res.sig@elementMetadata@listData %>% print

$type
[1] "intermediate" "results"      "results"      "results"      "results"     
[6] "results"     

$description
[1] "mean of normalized counts for all samples"     
[2] "log2 fold change (MLE): tissue Liver vs Kidney"
[3] "standard error: tissue Liver vs Kidney"        
[4] "Wald statistic: tissue Liver vs Kidney"        
[5] "Wald test p-value: tissue Liver vs Kidney"     
[6] "BH adjusted p-values"                          



In [164]:
res.sig@elementMetadata@listData %>% write_lines('description.txt')

In [162]:
getwd()

[1] "/project2/yangili1/cdai/SpliFi/code/test-ge"

In [166]:
system("cat description.txt", intern = T)

[1] "c(\"intermediate\", \"results\", \"results\", \"results\", \"results\", \"results\")"                                                                                                                                                                                 
[2] "c(\"mean of normalized counts for all samples\", \"log2 fold change (MLE): tissue Liver vs Kidney\", \"standard error: tissue Liver vs Kidney\", \"Wald statistic: tissue Liver vs Kidney\", \"Wald test p-value: tissue Liver vs Kidney\", \"BH adjusted p-values\")"

In [174]:
cbind(kidney[, 1:7], liver[, 3:7]) %>% 
fwrite("test_count.tsv", sep="\t")

In [176]:
names(coldata) <- c('sample_id', 'tissue')

In [177]:
coldata

sample_id,tissue
<chr>,<chr>
GTEX-11GS4-2326-SM-5A5KS,Kidney
GTEX-11OF3-1326-SM-5N9FJ,Kidney
GTEX-11TTK-1926-SM-5PNW8,Kidney
GTEX-12696-0926-SM-5FQTV,Kidney
GTEX-12WSG-0826-SM-5EQ5A,Kidney
GTEX-11DXY-0526-SM-5EGGQ,Liver
GTEX-11DXZ-0126-SM-5EGGY,Liver
GTEX-11EQ9-0526-SM-5A5JZ,Liver
GTEX-11GSP-0626-SM-5986T,Liver


In [179]:
fwrite(coldata, 'test-coldata.tsv', sep = '\t')

In [181]:
mcols(res)

DataFrame with 6 rows and 2 columns
                       type            description
                <character>            <character>
baseMean       intermediate mean of normalized c..
log2FoldChange      results log2 fold change (ML..
lfcSE               results standard error: tiss..
stat                results Wald statistic: tiss..
pvalue              results Wald test p-value: t..
padj                results   BH adjusted p-values

In [183]:
str(res)

Formal class 'DESeqResults' [package "DESeq2"] with 7 slots
  ..@ priorInfo      : list()
  ..@ rownames       : chr [1:22525] "ENSG00000227232.5" "ENSG00000268903.1" "ENSG00000269981.1" "ENSG00000241860.6" ...
  ..@ nrows          : int 22525
  ..@ elementType    : chr "ANY"
  ..@ elementMetadata:Formal class 'DFrame' [package "S4Vectors"] with 6 slots
  .. .. ..@ rownames       : NULL
  .. .. ..@ nrows          : int 6
  .. .. ..@ elementType    : chr "ANY"
  .. .. ..@ elementMetadata: NULL
  .. .. ..@ metadata       : list()
  .. .. ..@ listData       :List of 2
  .. .. .. ..$ type       : chr [1:6] "intermediate" "results" "results" "results" ...
  .. .. .. ..$ description: chr [1:6] "mean of normalized counts for all samples" "log2 fold change (MLE): tissue Liver vs Kidney" "standard error: tissue Liver vs Kidney" "Wald statistic: tissue Liver vs Kidney" ...
  ..@ metadata       :List of 6
  .. ..$ filterThreshold: Named num 2.64
  .. .. ..- attr(*, "names")= chr "0%"
  .. ..$ fil